<h1>Data prep and merge</h1>

<h2>Install required package/libraries and directory</h2>

In [7]:

# If not already installed, install pyreadstat (used to read .sas7bdat files)
!pip install pyreadstat
!pip install openpyxl

# Import core packages
import pandas as pd
import os
import numpy as np
import pyreadstat
from scipy.stats import sem
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore")


#path 
output_path = "/Users/dengshuyue/Desktop/SDOH/analysis/output/demo_summary.csv"
data_path = "/Users/dengshuyue/Desktop/SDOH/analysis/data"


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


<h2>Load and preview SAS dataset</h2>

In [8]:
# Path to your .sas7bdat file
file_path = "/Users/dengshuyue/Desktop/SDOH/analysis/data/less_important/demo_i.sas7bdat"

# Load the dataset using pandas (requires pyreadstat)
df = pd.read_sas(file_path, format="sas7bdat")

# Preview the data
pd.set_option('display.max_columns', None)
df.head()

# List all column names
print(df.columns.tolist())

['SEQN', 'SDDSRVYR', 'RIDSTATR', 'RIAGENDR', 'RIDAGEYR', 'RIDAGEMN', 'RIDRETH1', 'RIDRETH3', 'RIDEXMON', 'RIDEXAGM', 'DMQMILIZ', 'DMQADFC', 'DMDBORN4', 'DMDCITZN', 'DMDYRSUS', 'DMDEDUC3', 'DMDEDUC2', 'DMDMARTL', 'RIDEXPRG', 'SIALANG', 'SIAPROXY', 'SIAINTRP', 'FIALANG', 'FIAPROXY', 'FIAINTRP', 'MIALANG', 'MIAPROXY', 'MIAINTRP', 'AIALANGA', 'DMDHHSIZ', 'DMDFMSIZ', 'DMDHHSZA', 'DMDHHSZB', 'DMDHHSZE', 'DMDHRGND', 'DMDHRAGE', 'DMDHRBR4', 'DMDHREDU', 'DMDHRMAR', 'DMDHSEDU', 'WTINT2YR', 'WTMEC2YR', 'SDMVPSU', 'SDMVSTRA', 'INDHHIN2', 'INDFMIN2', 'INDFMPIR']


In [9]:

# Path to your .sas7bdat file
file_path = "/Users/dengshuyue/Desktop/SDOH/analysis/data/less_important/gg.sas7bdat"

# Load the dataset using pandas (requires pyreadstat)
df = pd.read_sas(file_path, format="sas7bdat")

# Preview the data
pd.set_option('display.max_columns', None)
df.head()

# List all column names
print(df.columns.tolist())


['SEQN', 'DAYS', 'DR12IFDC', 'WTDRD1', 'DR12DRST', 'SDDSRVYR', 'RIAGENDR', 'RIDAGEYR', 'RIDRETH1', 'DMDEDUC3', 'DMDEDUC2', 'INDFMPIR', 'DMDHREDU', 'WTINT2YR', 'WTMEC2YR', 'SDMVPSU', 'SDMVSTRA', 'DR12IKC2', 'CYCLE', 'WTDR2D', 'DR12FS', 'DMDHREDZ', 'age', 'race', 'edu', 'pedu', 'Incm', 'incm2', 'include', 'Weight16a', 'cycles', 'sex', 'age1', 'age2', 'age3', 'race2', 'race3', 'race4', 'weekend', '_NAME_', '_LABEL_', 'DRDAY1', 'DRDAY2', 'tkal1', 'tkal2', 'Tcal', 'DR12DAY', 'kcal1', 'kcal2', 'kcal3', 'kcal4', 'kcal12', 'wt1', 'wt2', 'wt3', 'wt4', 'wt12', 'pcte1', 'pcte2', 'pcte3', 'pcte4', 'pcte12', 'pctg1', 'pctg2', 'pctg3', 'pctg4', 'pctg12', 'kcals2', 'kcals5', 'kcals6', 'kcals9', 'kcals13', 'kcals14', 'kcals17', 'kcals20', 'kcals21', 'kcals22', 'kcals23', 'kcals25', 'kcals28', 'kcals29', 'kcals33', 'kcals36', 'kcals39', 'kcals41', 'kcals3', 'kcals37', 'kcals38', 'kcals40', 'kcals42', 'kcals1', 'kcals10', 'kcals16', 'kcals24', 'kcals15', 'kcals18', 'kcals7', 'kcals8', 'kcals19', 'kcals4

In [10]:
# Path to your .sas script file (SAS code, not dataset)
file_path = "/Users/dengshuyue/Desktop/SDOH/analysis/code/code_lu/Analysis1_COX_allcause.sas"

# Load the SAS script as plain text
with open(file_path, "r") as file:
    sas_code = file.read()

# Optionally preview the first 30 lines
sas_code_lines = sas_code.splitlines()
for line in sas_code_lines[:300]:
    print(line)



libname data "C:\Users\lwang18\Box\Projects\5_UPF_Mortality\data";
*%let home=C:\Users\LWANG18\Box\Projects\5_UPF_Mortality\results_revision ; 
%let home= C:\Users\lwang18\Box\Projects\Kroger project\Data for analysis\Scores;
 %let path= C:\Users\lwang18\Box\Projects\Kroger project\Data for analysis\Scores;

%let home= C:\Users\lwang18\OneDrive - Tufts\Desktop\Projects\Food Insecurity,;
libname out "C:\Users\lwang18\OneDrive - Tufts\Desktop\Projects\Food Insecurity,";

libname NHANES "C:\Users\LWANG18\Box\NHANES_Lu" ;

/** main analysis **/

%macro cox(data, dvar, evars, covars, death , out);
ods select all ; 
ODS OUTPUT PARAMETERESTIMATES=r0; 
proc surveyphreg data=&data;
	strata sdmvstra;
	cluster sdmvpsu;
	weight wt;
	class    sex (ref="1") race edu(ref="1") smk  pir(ref="3") SNAP(ref="0") FS ins2(ref="1") ins i_FCS_sdq hei2015q(ref="3") marriage  hoq065/param=ref;
	model py*&death(0)= &dvar &covars /rl ties=breslow;
run ;
data r0 ; set r0 ; 
HRCL=compress(round(hazardratio,0.01)||

In [4]:
# Display total number of rows (i.e., participants) in the dataset
print(f"Total sample size: {df.shape[0]:,}")

# Check the min and max age
min_age = df['RIDAGEYR'].min()
max_age = df['RIDAGEYR'].max()
print(f"Age range: {min_age} to {max_age} years")



Total sample size: 149,358
Age range: 0.0 to 85.0 years


<h2>Merge datasets step-by-step and track sample size</h2>

<h3>Merge demo</h3>

In [54]:
# %% Step 1: Set file path for NHANES demographic data
folder_path = "/Users/dengshuyue/Desktop/SDOH/analysis/data/less_important"


# Step 2: NHANES demographic files to include (1999–2018 cycles)
demo_files = [
    "demo",    # 1999–2000
    "demo_b",  # 2001–2002
    "demo_c",  # 2003–2004
    "demo_d",  # 2005–2006
    "demo_e",  # 2007–2008
    "demo_f",  # 2009–2010
    "demo_g",  # 2011–2012
    "demo_h",  # 2013–2014
    "demo_i",  # 2015–2016
    "demo_j"   # 2017–2018
]


# Step 3: Load and append files if they exist
demo_dfs = []
for f in demo_files:
    file_path = os.path.join(folder_path, f"{f}.sas7bdat")
    if os.path.exists(file_path):
        df = pd.read_sas(file_path)
        demo_dfs.append(df)
    else:
        print(f"⚠️ File not found: {file_path}")

# Step 4: Combine all loaded demographic datasets
demoall = pd.concat(demo_dfs, ignore_index=True).copy()

# Step 5: Filter to adults age 20+
# demoall = demoall[demoall["RIDAGEYR"] >= 20]

# Step 6: Recode marital status from DMDMARTL
conditions = [
    demoall['DMDMARTL'].isin([1, 6]),  # Married or living with partner
    demoall['DMDMARTL'].isin([3, 4]),  # Widowed or divorced
    demoall['DMDMARTL'] == 2,          # Never married
    demoall['DMDMARTL'] == 5           # Separated
]
choices = [1, 2, 3, 4]
demoall['marriage'] = np.select(conditions, choices, default=pd.NA)

# Step 7: Keep only relevant variables
# demoall = demoall[['SEQN', 'marriage', 'SDDSRVYR']]

demoall = demoall[['SEQN', 'RIDAGEYR', 'marriage', 'SDDSRVYR']]


# Step 8: Summary
print(f"✅ Total rows: {demoall.shape[0]}") # (age 20+)
print(f"✅ Unique SEQN values: {demoall['SEQN'].nunique()}")

# Optional: Preview first few rows
# print(demoall.head())

# Step 9: Save cleaned demoall for reuse
demoall.to_pickle("/Users/dengshuyue/Desktop/SDOH/analysis/data/demoall.pkl")  # Fast & preserves types
# Or as CSV if needed:
demoall.to_csv("/Users/dengshuyue/Desktop/SDOH/analysis/data/demoall.csv", index=False)



✅ Total rows: 101316
✅ Unique SEQN values: 101316


In [55]:
import pandas as pd
from pathlib import Path
# --- Paths -------------------------------------------------------
root = Path("/Users/dengshuyue/Desktop/SDOH/analysis")
mort_path = root / "data" / "less_important" / "mortality9918.sas7bdat"

# --- Load mortality (1999–2018) --------------------------------
mort = pd.read_sas(mort_path, format="sas7bdat", encoding="latin1")
mort.columns = mort.columns.str.upper()

keep_cols = [c for c in mort.columns if c in {
    "SEQN","MORTSTAT","PERMTH_INT","PERMTH_EXM","UCOD_LEADING","DIABETES","HYPERTEN","DODQTR","DODYEAR"
}]
if keep_cols:
    mort = mort[keep_cols]

# --- 1) Ensure demoall columns & types --------------------------
demoall.columns = demoall.columns.str.upper()
if "SDDSRVYR" not in demoall.columns:
    raise ValueError("SDDSRVYR not found in demoall. Ensure 1999–2018 DEMO files include it.")
# Sometimes SDDSRVYR is object/string; coerce to numeric
demoall["SDDSRVYR"] = pd.to_numeric(demoall["SDDSRVYR"], errors="coerce")

# --- 2) Merge age + cycle into mortality ------------------------
mort_with_demo = mort.merge(
    demoall[["SEQN", "RIDAGEYR", "SDDSRVYR"]],
    on="SEQN", how="left"
)

# --- 3) Filter adults (≥20 years) -------------------------------
# mort_with_demo = mort_with_demo.dropna(subset=["RIDAGEYR"])
# mort_with_demo = mort_with_demo[mort_with_demo["RIDAGEYR"] >= 20]

# --- 4) Cycle counts among adults 20+ ---------------------------
# cycle_counts = mort_with_demo["SDDSRVYR"].value_counts(dropna=False).sort_index()
# print("📊 NHANES cycles in mortality data (age 20+):")
# print(cycle_counts)
# print(f"Total valid mortality count (age 20+): {int(cycle_counts.sum())}")

# Optional: map cycle codes to years for readability
cycle_map = {
    1: "1999–2000", 2: "2001–2002", 3: "2003–2004",
    4: "2005–2006", 5: "2007–2008", 6: "2009–2010",
    7: "2011–2012", 8: "2013–2014", 9: "2015–2016",
    10: "2017–2018"
}
print("\nReadable cycles:")
print(cycle_counts.rename(index=cycle_map))



Readable cycles:
SDDSRVYR
1999–2000    4877
2001–2002    5405
2003–2004    5031
2005–2006    4977
2007–2008    5926
2009–2010    6201
2011–2012    5546
2013–2014    5757
2015–2016    5701
2017–2018    5524
Name: count, dtype: int64


In [56]:
# print(mort_with_demo["SDDSRVYR"].min())
# print(mort_with_demo["SDDSRVYR"].max())

# SEQN 
print("Min:", mort_with_demo["SEQN"].min())
print("Min:", mort_with_demo["SEQN"].max())


Min: 2.0
Min: 102956.0


In [57]:
# Make sure both SEQN are numeric
mort["SEQN"] = pd.to_numeric(mort["SEQN"], errors="coerce")
demoall["SEQN"] = pd.to_numeric(demoall["SEQN"], errors="coerce")

# Check min/max ranges
print("mort SEQN range:", mort["SEQN"].min(), mort["SEQN"].max())
print("demoall SEQN range:", demoall["SEQN"].min(), demoall["SEQN"].max())

# Check overlap
overlap = set(mort["SEQN"]).intersection(set(demoall["SEQN"]))
print("Number of overlapping SEQNs:", len(overlap))

# If very small, print a sample
if len(overlap) > 0:
    print("Example overlapping IDs:", list(overlap)[:10])
else:
    print("⚠️ No overlap found — SEQNs don't match between mort and demoall")


mort SEQN range: 2.0 102956.0
demoall SEQN range: 1.0 102956.0
Number of overlapping SEQNs: 59064
Example overlapping IDs: [2.0, 5.0, 6.0, 7.0, 10.0, 12.0, 13.0, 14.0, 15.0, 16.0]


<h3>Preprocess SDOH</h3>

In [71]:

# --- Employment data (OCQ) ---
ocq = pd.read_sas(f"{folder_path}/ocq.sas7bdat")

ocq['employ'] = np.nan
ocq.loc[ocq['OCD150'] == 1, 'employ'] = 1
ocq.loc[(ocq['OCD150'] == 3) | (ocq['OCQ380'] == 5), 'employ'] = 2
ocq.loc[ocq['OCQ380'] == 3, 'employ'] = 3
ocq.loc[ocq['OCQ380'].isin([4, 6]), 'employ'] = 4
ocq.loc[ocq['OCQ380'].isin([1, 2, 7]), 'employ'] = 5
ocq['unemployment'] = (ocq['employ'] == 2).astype(int)
ocq = ocq[['SEQN', 'employ', 'unemployment']]

# --- Housing data (HOQ) ---
hoq = pd.read_sas(f"{folder_path}/hoq.sas7bdat")
hoq.loc[hoq['HOQ065'].isin([7, 9]), 'HOQ065'] = np.nan
hoq = hoq[['SEQN', 'HOD050', 'HOQ065']]

# --- Health insurance (HIQs) ---
hiqs = pd.read_sas(f"{folder_path}/hiqs.sas7bdat")
ins = pd.DataFrame({'SEQN': hiqs['SEQN']})
ins['ins'] = np.nan

# Private insurance
ins.loc[(hiqs['HIQ031A'] == 14) | (hiqs['HID030A'] == 1), 'ins'] = 1

# Medicare
ins.loc[
    ((hiqs['HIQ031B'] == 15) & (hiqs['HIQ031D'] != 17) & (hiqs['HIQ031E'] != 18)) |
    ((hiqs['HID030B'] == 1) & (hiqs['HID030C'] != 1)),
    'ins'
] = 2

# Medicaid
ins.loc[
    (((hiqs['HIQ031D'] == 17) | (hiqs['HIQ031E'] == 18)) & (hiqs['HIQ031B'] != 15)) |
    ((hiqs['HID030B'] != 1) & (hiqs['HID030C'] == 1)),
    'ins'
] = 3
ins.loc[
    ((hiqs['HIQ031B'] == 15) & (hiqs['HIQ031D'] == 17)) |
    ((hiqs['HID030B'] == 1) & (hiqs['HID030C'] == 1)),
    'ins'
] = 3

# Other insurance
ins.loc[
    (hiqs[['HIQ031C', 'HIQ031F', 'HIQ031G', 'HIQ031H', 'HIQ031I']].eq(1).any(axis=1)) |
    (hiqs['HID030D'] == 1),
    'ins'
] = 5

# No insurance
ins.loc[(hiqs['HIQ011'] == 2) | (hiqs['HID010'] == 2), 'ins'] = 0

# --- SNAP and Food Security (FSQS) ---
fsqs = pd.read_sas(f"{folder_path}/fsqs.sas7bdat")
snap = pd.DataFrame({'SEQN': fsqs['SEQN'], 'FSDHH': fsqs['FSDHH']})

snap['SNAP'] = np.nan
snap.loc[fsqs['FSQ165'] == 2, 'SNAP'] = 0
snap.loc[fsqs['FSQ012'] == 1, 'SNAP'] = 1
snap.loc[fsqs['FSQ012'] == 2, 'SNAP'] = 0
snap.loc[fsqs['FSQ171'] == 1, 'SNAP'] = 1
snap.loc[fsqs['FSQ171'] == 2, 'SNAP'] = 0
snap.loc[fsqs['FSD170N'] >= 1, 'SNAP'] = 1
snap.loc[fsqs['FSQ170'] == 1, 'SNAP'] = 1
snap.loc[(fsqs['FSQ170'] == 2) & (fsqs['FSD170N'] < 1), 'SNAP'] = 0
snap.loc[fsqs['FSD200'] == 1, 'SNAP'] = 1

# Food Security
snap['FS'] = np.nan
snap.loc[fsqs['FSDHH'].isin([1, 2]), 'FS'] = 1
snap.loc[fsqs['FSDHH'] > 2, 'FS'] = 0

# Keep only final columns
snap = snap[['SEQN', 'SNAP', 'FSDHH', 'FS']]


In [72]:
print(ocq["SEQN"])


0         21005.0
1         21006.0
2         21008.0
3         21009.0
4         21010.0
           ...   
50909    102950.0
50910    102952.0
50911    102953.0
50912    102954.0
50913    102956.0
Name: SEQN, Length: 50914, dtype: float64


In [74]:
print(demoall["SEQN"])

0              1.0
1              2.0
2              3.0
3              4.0
4              5.0
            ...   
101311    102952.0
101312    102953.0
101313    102954.0
101314    102955.0
101315    102956.0
Name: SEQN, Length: 101316, dtype: float64


Found 14 matching file(s):

[MOVE] /Users/dengshuyue/Desktop/SDOH/analysis/data/less_important/demo_d.sas7bdat  ->  /Users/dengshuyue/Desktop/SDOH/analysis/data/nhanes_by_module/demographic/demo_d.sas7bdat
[MOVE] /Users/dengshuyue/Desktop/SDOH/analysis/data/less_important/demo_e.sas7bdat  ->  /Users/dengshuyue/Desktop/SDOH/analysis/data/nhanes_by_module/demographic/demo_e.sas7bdat
[MOVE] /Users/dengshuyue/Desktop/SDOH/analysis/data/less_important/demo.sas7bdat  ->  /Users/dengshuyue/Desktop/SDOH/analysis/data/nhanes_by_module/demographic/demo.sas7bdat
[MOVE] /Users/dengshuyue/Desktop/SDOH/analysis/data/less_important/demo_c.sas7bdat  ->  /Users/dengshuyue/Desktop/SDOH/analysis/data/nhanes_by_module/demographic/demo_c.sas7bdat
[MOVE] /Users/dengshuyue/Desktop/SDOH/analysis/data/less_important/demo_b.sas7bdat  ->  /Users/dengshuyue/Desktop/SDOH/analysis/data/nhanes_by_module/demographic/demo_b.sas7bdat
[MOVE] /Users/dengshuyue/Desktop/SDOH/analysis/data/less_important/demo_i.sas7bdat  ->

In [88]:
import pandas as pd
import numpy as np
from pathlib import Path
import os

# -----------------------------
# Step 0: Setup (NEW PATHS)
# -----------------------------
base_module_path = Path("/Users/dengshuyue/Desktop/SDOH/analysis/data/nhanes_by_module")

ocq_path = base_module_path / "ocq" / "ocq.sas7bdat"     # main OCQ (2003–2018)
hoq_path = base_module_path / "hoq" / "hoq.sas7bdat"     # main HOQ (2003–2018)
hiq_path = base_module_path / "hiq" / "hiqs.sas7bdat"    # insurance (HIQS for later years)
fsq_path = base_module_path / "fsq" / "fsqs.sas7bdat"    # food security (FSQS for later years)

# Step 1: Load demoall (unchanged)
demoall = pd.read_pickle("/Users/dengshuyue/Desktop/SDOH/analysis/data/demoall.pkl")
if "SEQN" not in demoall.columns or "RIDAGEYR" not in demoall.columns:
    raise ValueError("demoall is missing SEQN or RIDAGEYR")

# Step 2: Filter demoall to adults (age ≥ 20)
age_df = demoall[["SEQN", "RIDAGEYR"]].dropna()
age_df = age_df[age_df["RIDAGEYR"] >= 20]

# Helper: Filter any df to age ≥ 20 using demoall
def filter_adults(df):
    return df.merge(age_df, on="SEQN", how="inner")

# -----------------------------
# Step 3: Employment (OCQ)
# -----------------------------
if not ocq_path.exists():
    raise FileNotFoundError(f"OCQ file not found at: {ocq_path}")

ocq = filter_adults(pd.read_sas(str(ocq_path), format="sas7bdat", encoding="latin1"))
ocq['employ'] = np.nan
ocq.loc[ocq['OCD150'] == 1, 'employ'] = 1
ocq.loc[(ocq['OCD150'] == 3) | (ocq['OCQ380'] == 5), 'employ'] = 2
ocq.loc[ocq['OCQ380'] == 3, 'employ'] = 3
ocq.loc[ocq['OCQ380'].isin([4, 6]), 'employ'] = 4
ocq.loc[ocq['OCQ380'].isin([1, 2, 7]), 'employ'] = 5
ocq['unemployment'] = (ocq['employ'] == 2).astype(int)
ocq = ocq[['SEQN', 'employ', 'unemployment']]

# -----------------------------
# Step 4: Housing (HOQ)
# -----------------------------
if not hoq_path.exists():
    raise FileNotFoundError(f"HOQ file not found at: {hoq_path}")

hoq = filter_adults(pd.read_sas(str(hoq_path), format="sas7bdat", encoding="latin1"))
if "HOQ065" in hoq.columns:
    hoq.loc[hoq['HOQ065'].isin([7, 9]), 'HOQ065'] = np.nan
keep_hoq = [c for c in ['SEQN', 'HOD050', 'HOQ065'] if c in hoq.columns]
hoq = hoq[keep_hoq]

# -----------------------------
# Step 5: Insurance (HIQS)
# -----------------------------
if not hiq_path.exists():
    raise FileNotFoundError(f"HIQS file not found at: {hiq_path}")

hiqs = filter_adults(pd.read_sas(str(hiq_path), format="sas7bdat", encoding="latin1"))
ins = pd.DataFrame({'SEQN': hiqs['SEQN']})
ins['ins'] = np.nan

# Private
if 'HIQ031A' in hiqs: ins.loc[(hiqs['HIQ031A'] == 14), 'ins'] = 1
if 'HID030A' in hiqs: ins.loc[(hiqs['HID030A'] == 1), 'ins'] = 1

# Medicare
cond_med = False
if set(['HIQ031B','HIQ031D','HIQ031E']).issubset(hiqs.columns):
    cond_med = ((hiqs['HIQ031B'] == 15) & (hiqs['HIQ031D'] != 17) & (hiqs['HIQ031E'] != 18))
if set(['HID030B','HID030C']).issubset(hiqs.columns):
    cond_med = cond_med | ((hiqs['HID030B'] == 1) & (hiqs['HID030C'] != 1)) if isinstance(cond_med, pd.Series) else ((hiqs['HID030B'] == 1) & (hiqs['HID030C'] != 1))
ins.loc[cond_med, 'ins'] = 2

# Medicaid
cond_mcaid = False
if set(['HIQ031B','HIQ031D','HIQ031E']).issubset(hiqs.columns):
    cond_mcaid = (((hiqs['HIQ031D'] == 17) | (hiqs['HIQ031E'] == 18)) & (hiqs['HIQ031B'] != 15))
if set(['HID030B','HID030C']).issubset(hiqs.columns):
    cond_mcaid = cond_mcaid | ((hiqs['HID030B'] != 1) & (hiqs['HID030C'] == 1)) if isinstance(cond_mcaid, pd.Series) else ((hiqs['HID030B'] != 1) & (hiqs['HID030C'] == 1))
ins.loc[cond_mcaid, 'ins'] = 3

# Medicaid (both present)
if set(['HIQ031B','HIQ031D']).issubset(hiqs.columns):
    ins.loc[((hiqs['HIQ031B'] == 15) & (hiqs['HIQ031D'] == 17)), 'ins'] = 3
if set(['HID030B','HID030C']).issubset(hiqs.columns):
    ins.loc[((hiqs['HID030B'] == 1) & (hiqs['HID030C'] == 1)), 'ins'] = 3

# Other insurance
cols_other = [c for c in ['HIQ031C','HIQ031F','HIQ031G','HIQ031H','HIQ031I'] if c in hiqs.columns]
cond_other = hiqs[cols_other].eq(1).any(axis=1) if cols_other else False
if 'HID030D' in hiqs:
    cond_other = cond_other | (hiqs['HID030D'] == 1) if isinstance(cond_other, pd.Series) else (hiqs['HID030D'] == 1)
ins.loc[cond_other, 'ins'] = 5

# No insurance
conds_none = []
if 'HIQ011' in hiqs: conds_none.append(hiqs['HIQ011'] == 2)
if 'HID010' in hiqs: conds_none.append(hiqs['HID010'] == 2)
if conds_none:
    ins.loc[np.logical_or.reduce(conds_none), 'ins'] = 0


# -----------------------------
# Step 6: SNAP & Food Security (FSQS)
# -----------------------------
if not fsq_path.exists():
    raise FileNotFoundError(f"FSQS file not found: {fsq_path}")

fsqs = filter_adults(pd.read_sas(str(fsq_path), format="sas7bdat", encoding="latin1"))
snap = pd.DataFrame({'SEQN': fsqs['SEQN']})
if 'FSDHH' in fsqs: snap['FSDHH'] = fsqs['FSDHH']

snap['SNAP'] = np.nan
if 'FSQ165' in fsqs: snap.loc[fsqs['FSQ165'] == 2, 'SNAP'] = 0
if 'FSQ012' in fsqs:
    snap.loc[fsqs['FSQ012'] == 1, 'SNAP'] = 1
    snap.loc[fsqs['FSQ012'] == 2, 'SNAP'] = 0
if 'FSQ171' in fsqs:
    snap.loc[fsqs['FSQ171'] == 1, 'SNAP'] = 1
    snap.loc[fsqs['FSQ171'] == 2, 'SNAP'] = 0
if 'FSD170N' in fsqs: snap.loc[fsqs['FSD170N'] >= 1, 'SNAP'] = 1
if 'FSQ170' in fsqs:
    snap.loc[fsqs['FSQ170'] == 1, 'SNAP'] = 1
    snap.loc[(fsqs['FSQ170'] == 2) & (fsqs.get('FSD170N', pd.Series(index=fsqs.index)) < 1), 'SNAP'] = 0
if 'FSD200' in fsqs: snap.loc[fsqs['FSD200'] == 1, 'SNAP'] = 1

snap['FS'] = np.nan
if 'FSDHH' in fsqs:
    snap.loc[fsqs['FSDHH'].isin([1, 2]), 'FS'] = 1
    snap.loc[fsqs['FSDHH'] > 2, 'FS'] = 0

snap = snap[['SEQN', 'SNAP', 'FSDHH', 'FS']]


In [91]:

# check
print(hiqs["SEQN"])

print(snap["SEQN"])

0         21009.0
1         21010.0
2         21012.0
3         21015.0
4         21017.0
           ...   
44785    102950.0
44786    102952.0
44787    102953.0
44788    102954.0
44789    102956.0
Name: SEQN, Length: 44790, dtype: float64
0         21009.0
1         21010.0
2         21012.0
3         21015.0
4         21017.0
           ...   
44785    102950.0
44786    102952.0
44787    102953.0
44788    102954.0
44789    102956.0
Name: SEQN, Length: 44790, dtype: float64


<h4>Add previous years SDOH</h4>

In [79]:

# merge previous 99-03 ir 99-01  first check their SEQN

import pyreadstat
from pathlib import Path

ocq_dir = Path("/Users/dengshuyue/Desktop/SDOH/analysis/data/nhanes_by_module/ocq")

files = [
    ("OCQ.xpt",   "1999–2000"),
    ("OCQ_B.xpt", "2001–2002"),
    ("OCQ_C.xpt", "2003–2004"),
]

for fname, years in files:
    path = ocq_dir / fname
    if not path.exists():
        print(f"⚠️ File not found: {path}")
        continue
    
    df, meta = pyreadstat.read_xport(path)
    df.columns = df.columns.str.upper()
    
    print(f"\n✅ Loaded {fname} ({years})")
    print("Shape:", df.shape)
    print("SEQN min:", df['SEQN'].min(), "SEQN max:", df['SEQN'].max())
    print("Unique SEQN:", df['SEQN'].nunique())
    print("First 5 SEQN:", df['SEQN'].head().tolist())



✅ Loaded OCQ.xpt (1999–2000)
Shape: (7295, 36)
SEQN min: 2.0 SEQN max: 9965.0
Unique SEQN: 7295
First 5 SEQN: [2.0, 5.0, 6.0, 7.0, 8.0]

✅ Loaded OCQ_B.xpt (2001–2002)
Shape: (6634, 25)
SEQN min: 9966.0 SEQN max: 21004.0
Unique SEQN: 6634
First 5 SEQN: [9966.0, 9967.0, 9968.0, 9969.0, 9970.0]

✅ Loaded OCQ_C.xpt (2003–2004)
Shape: (6213, 25)
SEQN min: 21005.0 SEQN max: 31125.0
Unique SEQN: 6213
First 5 SEQN: [21005.0, 21006.0, 21008.0, 21009.0, 21010.0]


In [82]:
# first start with ocq 
import pandas as pd
import numpy as np
from pathlib import Path
import pyreadstat

# --- Paths ---
ocq_dir = Path("/Users/dengshuyue/Desktop/SDOH/analysis/data/nhanes_by_module/ocq")
ocq_main_path = ocq_dir / "ocq.sas7bdat"                 # 2003–2018 consolidated
ocq_early_paths = [(ocq_dir/"OCQ.xpt", 1), (ocq_dir/"OCQ_B.xpt", 2)]  # 1999–2002

def recode_employment(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = df.columns.str.upper()
    df["EMPLOY"] = np.nan
    if "OCD150" in df:
        df.loc[df["OCD150"] == 1, "EMPLOY"] = 1
        df.loc[df["OCD150"] == 3, "EMPLOY"] = 2
    if "OCQ380" in df:
        df.loc[df["OCQ380"] == 5, "EMPLOY"] = 2
        df.loc[df["OCQ380"] == 3, "EMPLOY"] = 3
        df.loc[df["OCQ380"].isin([4, 6]), "EMPLOY"] = 4
        df.loc[df["OCQ380"].isin([1, 2, 7]), "EMPLOY"] = 5
    df["UNEMPLOYMENT"] = (df["EMPLOY"] == 2).astype(int)
    keep = [c for c in ["SEQN","EMPLOY","UNEMPLOYMENT","SDDSRVYR"] if c in df.columns]
    return df[keep]

# --- A) Read early cycles (1999–2002) ---
early_parts = []
for p, cyc in ocq_early_paths:
    if p.exists():
        df, _ = pyreadstat.read_xport(str(p))
        df.columns = df.columns.str.upper()
        df["SDDSRVYR"] = cyc
        early_parts.append(recode_employment(df))
    else:
        print(f"⚠️ Missing early OCQ file: {p}")
ocq_early = pd.concat(early_parts, ignore_index=True) if early_parts else pd.DataFrame()

# --- B) Read main consolidated OCQ (2003–2018) from the SAME folder ---
if not ocq_main_path.exists():
    raise FileNotFoundError(f"Main OCQ not found: {ocq_main_path}")
ocq_main_raw = pd.read_sas(str(ocq_main_path), format="sas7bdat", encoding="latin1")
ocq_main_raw.columns = ocq_main_raw.columns.str.upper()

# Optional adult filter if you have it defined
try:
    ocq_main_raw = filter_adults(ocq_main_raw)
except NameError:
    pass

# If SDDSRVYR isn't present in the main file, you can skip it or infer later via SEQN.
ocq_main = recode_employment(ocq_main_raw)

# --- C) Combine to final OCQ (1999–2018) ---
ocq = pd.concat([ocq_early, ocq_main], ignore_index=True)

# Hygiene
ocq["SEQN"] = pd.to_numeric(ocq["SEQN"], errors="coerce").astype("Int64")
if "SDDSRVYR" in ocq.columns:
    ocq["SDDSRVYR"] = pd.to_numeric(ocq["SDDSRVYR"], errors="coerce").astype("Int64")
ocq = ocq.dropna(subset=["SEQN"]).drop_duplicates(subset=["SEQN"])

# --- D) Quick checks ---
print("Final OCQ shape:", ocq.shape)
print("SEQN range:", ocq["SEQN"].min(), "→", ocq["SEQN"].max())
if "SDDSRVYR" in ocq.columns:
    print("Cycles present:\n", ocq["SDDSRVYR"].value_counts().sort_index())


Final OCQ shape: (58719, 4)
SEQN range: 2 → 102956
Cycles present:
 SDDSRVYR
1    7295
2    6634
Name: count, dtype: Int64


In [92]:
# hoq (housing)

import pandas as pd
import numpy as np
from pathlib import Path

# --- Paths ---
hoq_dir = Path("/Users/dengshuyue/Desktop/SDOH/analysis/data/nhanes_by_module/hoq")
hoq_main_path = hoq_dir / "hoq.sas7bdat"  # your 2003–2018 consolidated file
early_candidates = [
    (hoq_dir / "HOQ.xpt",   1),  # 1999–2000
    (hoq_dir / "HOQ_B.xpt", 2),  # 2001–2002
    # if you happen to have SAS versions instead of XPTs, we’ll try those too:
    (hoq_dir / "hoq.sas7bdat",   1),  # fallback for 99–00 (rare)
    (hoq_dir / "hoq_b.sas7bdat", 2),
]

def read_any(path: Path) -> pd.DataFrame:
    if path.suffix.lower() == ".xpt":
        import pyreadstat
        df, _ = pyreadstat.read_xport(str(path))
    elif path.suffix.lower() == ".sas7bdat":
        df = pd.read_sas(str(path), format="sas7bdat", encoding="latin1")
    else:
        raise ValueError(f"Unsupported file: {path}")
    df.columns = df.columns.str.upper()
    return df

def preprocess_hoq(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    # Clean HOQ065: set 7/9 to NaN where present
    if "HOQ065" in df.columns:
        df.loc[df["HOQ065"].isin([7, 9]), "HOQ065"] = np.nan
    # Keep the core columns that exist
    keep = [c for c in ["SEQN", "HOD050", "HOQ065", "SDDSRVYR"] if c in df.columns]
    return df[keep]

# --- A) Early cycles (99–02) ---
early_parts = []
seen_cycles = set()
for p, cyc in early_candidates:
    if p.exists() and cyc not in seen_cycles:
        df = read_any(p)
        df["SDDSRVYR"] = cyc
        early_parts.append(preprocess_hoq(df))
        seen_cycles.add(cyc)
hoq_early = pd.concat(early_parts, ignore_index=True) if early_parts else pd.DataFrame()

# --- B) Main consolidated HOQ (2003–2018) ---
if not hoq_main_path.exists():
    raise FileNotFoundError(f"Main HOQ not found: {hoq_main_path}")
hoq_main_raw = read_any(hoq_main_path)
# (Optional) If you have an adult filter function:
try:
    hoq_main_raw = filter_adults(hoq_main_raw)  # your function, if defined
except NameError:
    pass
hoq_main = preprocess_hoq(hoq_main_raw)

# --- C) Combine to final HOQ (1999–2018) ---
hoq_all = pd.concat([hoq_early, hoq_main], ignore_index=True)

# Hygiene
if "SEQN" in hoq_all:
    hoq_all["SEQN"] = pd.to_numeric(hoq_all["SEQN"], errors="coerce").astype("Int64")
if "SDDSRVYR" in hoq_all:
    hoq_all["SDDSRVYR"] = pd.to_numeric(hoq_all["SDDSRVYR"], errors="coerce").astype("Int64")
hoq_all = hoq_all.dropna(subset=["SEQN"]).drop_duplicates(subset=["SEQN"])

# --- D) Quick checks ---
print("Final HOQ shape:", hoq_all.shape)
print("SEQN range:", hoq_all["SEQN"].min(), "→", hoq_all["SEQN"].max())
if "SDDSRVYR" in hoq_all:
    cycle_map = {1:"1999–2000",2:"2001–2002",3:"2003–2004",4:"2005–2006",5:"2007–2008",
                 6:"2009–2010",7:"2011–2012",8:"2013–2014",9:"2015–2016",10:"2017–2018"}
    counts = hoq_all["SDDSRVYR"].value_counts().sort_index()
    print("Cycles present:\n", counts.rename(index=cycle_map))

# (Optional) Save for reuse
hoq_all.to_csv("/Users/dengshuyue/Desktop/SDOH/analysis/data/nhanes_by_module/hoq/hoq_99_18.csv", index=False)


Final HOQ shape: (65794, 4)
SEQN range: 1 → 102956
Cycles present:
 SDDSRVYR
1999–2000     9965
2001–2002    11039
Name: count, dtype: Int64


In [99]:
# hiqs 

# -----------------------------
# Insurance (HIQ/HIQS) — add 1999–2002
# -----------------------------
from pathlib import Path
import pandas as pd
import numpy as np

base_module_path = Path("/Users/dengshuyue/Desktop/SDOH/analysis/data/nhanes_by_module")
hiq_dir = base_module_path / "hiq"

# Main consolidated later years (HIQS for 2003–2018)
hiq_main_path = hiq_dir / "hiqs.sas7bdat"

# Early cycles (prefer XPT; fallback to sas7bdat if that’s what you have)
early_candidates = [
    (hiq_dir / "HIQ.xpt",       1),  # 1999–2000
    (hiq_dir / "HIQ_B.xpt",     2),  # 2001–2002
    (hiq_dir / "hiq.sas7bdat",  1),  # fallback (rare)
    (hiq_dir / "hiq_b.sas7bdat",2),
]

def read_any(p: Path) -> pd.DataFrame:
    if p.suffix.lower() == ".xpt":
        import pyreadstat
        df, _ = pyreadstat.read_xport(str(p))
    elif p.suffix.lower() == ".sas7bdat":
        df = pd.read_sas(str(p), format="sas7bdat", encoding="latin1")
    else:
        raise ValueError(f"Unsupported file: {p}")
    df.columns = df.columns.str.upper()
    return df

# A) Early cycles (99–02)
early_parts, seen = [], set()
for p, cyc in early_candidates:
    if p.exists() and cyc not in seen:
        df = read_any(p)
        df["SDDSRVYR"] = cyc
        early_parts.append(df)
        seen.add(cyc)
hiq_early_raw = pd.concat(early_parts, ignore_index=True) if early_parts else pd.DataFrame()

# B) Main consolidated (03–18)
if not hiq_main_path.exists():
    raise FileNotFoundError(f"HIQS file not found at: {hiq_main_path}")
hiq_main_raw = read_any(hiq_main_path)

# C) Filter to adults using your helper
hiq_early = filter_adults(hiq_early_raw) if not hiq_early_raw.empty else hiq_early_raw
hiqs       = filter_adults(hiq_main_raw)

# D) Stack early + main
hiq_all = pd.concat([hiq_early, hiqs], ignore_index=True)
hiq_all.columns = hiq_all.columns.str.upper()

# E) Compute insurance 'ins' on the combined data (guards for varying columns)
ins = pd.DataFrame({"SEQN": hiq_all["SEQN"]})
if "SDDSRVYR" in hiq_all.columns:
    ins["SDDSRVYR"] = hiq_all["SDDSRVYR"]
ins["ins"] = np.nan

# Private
if "HIQ031A" in hiq_all: ins.loc[hiq_all["HIQ031A"] == 14, "ins"] = 1
if "HID030A" in hiq_all: ins.loc[hiq_all["HID030A"] == 1,  "ins"] = 1

# Medicare
cond_med = False
if {"HIQ031B","HIQ031D","HIQ031E"}.issubset(hiq_all.columns):
    cond_med = (hiq_all["HIQ031B"] == 15) & (hiq_all["HIQ031D"] != 17) & (hiq_all["HIQ031E"] != 18)
if {"HID030B","HID030C"}.issubset(hiq_all.columns):
    cond_med = cond_med | ((hiq_all["HID030B"] == 1) & (hiq_all["HID030C"] != 1)) if isinstance(cond_med, pd.Series) else ((hiq_all["HID030B"] == 1) & (hiq_all["HID030C"] != 1))
ins.loc[cond_med, "ins"] = 2

# Medicaid
cond_mcaid = False
if {"HIQ031B","HIQ031D","HIQ031E"}.issubset(hiq_all.columns):
    cond_mcaid = (((hiq_all["HIQ031D"] == 17) | (hiq_all["HIQ031E"] == 18)) & (hiq_all["HIQ031B"] != 15))
if {"HID030B","HID030C"}.issubset(hiq_all.columns):
    cond_mcaid = cond_mcaid | ((hiq_all["HID030B"] != 1) & (hiq_all["HID030C"] == 1)) if isinstance(cond_mcaid, pd.Series) else ((hiq_all["HID030B"] != 1) & (hiq_all["HID030C"] == 1))
ins.loc[cond_mcaid, "ins"] = 3

# Medicaid when both present
if {"HIQ031B","HIQ031D"}.issubset(hiq_all.columns):
    ins.loc[(hiq_all["HIQ031B"] == 15) & (hiq_all["HIQ031D"] == 17), "ins"] = 3
if {"HID030B","HID030C"}.issubset(hiq_all.columns):
    ins.loc[(hiq_all["HID030B"] == 1) & (hiq_all["HID030C"] == 1), "ins"] = 3

# Other insurance
other_cols = [c for c in ["HIQ031C","HIQ031F","HIQ031G","HIQ031H","HIQ031I"] if c in hiq_all.columns]
cond_other = hiq_all[other_cols].eq(1).any(axis=1) if other_cols else False
if "HID030D" in hiq_all:
    cond_other = cond_other | (hiq_all["HID030D"] == 1) if isinstance(cond_other, pd.Series) else (hiq_all["HID030D"] == 1)
ins.loc[cond_other, "ins"] = 5

# No insurance
conds_none = []
if "HIQ011" in hiq_all: conds_none.append(hiq_all["HIQ011"] == 2)
if "HID010" in hiq_all: conds_none.append(hiq_all["HID010"] == 2)
if conds_none:
    ins.loc[np.logical_or.reduce(conds_none), "ins"] = 0

# Tidy up
ins["SEQN"] = pd.to_numeric(ins["SEQN"], errors="coerce").astype("Int64")
if "SDDSRVYR" in ins.columns:
    ins["SDDSRVYR"] = pd.to_numeric(ins["SDDSRVYR"], errors="coerce").astype("Int64")
ins = ins.dropna(subset=["SEQN"]).drop_duplicates(subset=["SEQN"])

print("Insurance table shape:", ins.shape)
if "SDDSRVYR" in ins.columns:
    print("Cycles present:\n", ins["SDDSRVYR"].value_counts().sort_index())

# print(ins["SEQN"])

# (Optional) Save for reuse
ins.to_csv("/Users/dengshuyue/Desktop/SDOH/analysis/data/nhanes_by_module/hiq/ins_99_18.csv", index=False)


Insurance table shape: (55081, 3)
Cycles present:
 SDDSRVYR
1    4880
2    5411
Name: count, dtype: Int64


In [101]:
# fsq
# -----------------------------
# Step 6: SNAP & Food Security (FSQ/FSQS) — add 1999–2002
# -----------------------------
from pathlib import Path
import pandas as pd
import numpy as np

base_module_path = Path("/Users/dengshuyue/Desktop/SDOH/analysis/data/nhanes_by_module")
fsq_dir = base_module_path / "fsq"

# Main later-years file (FSQS for 2003–2018)
fsq_main_path = fsq_dir / "fsqs.sas7bdat"

# Early cycles (prefer XPT; fallback to sas7bdat if that’s what you have)
early_candidates = [
    (fsq_dir / "FSQ.xpt",         1),  # 1999–2000
    (fsq_dir / "FSQ_B.xpt",       2),  # 2001–2002
    (fsq_dir / "fsq.sas7bdat",    1),  # fallback (rare)
    (fsq_dir / "fsq_b.sas7bdat",  2),
]

def read_any(p: Path) -> pd.DataFrame:
    if p.suffix.lower() == ".xpt":
        import pyreadstat
        df, _ = pyreadstat.read_xport(str(p))
    elif p.suffix.lower() == ".sas7bdat":
        df = pd.read_sas(str(p), format="sas7bdat", encoding="latin1")
    else:
        raise ValueError(f"Unsupported file: {p}")
    df.columns = df.columns.str.upper()
    return df

# A) Early cycles (99–02)
early_parts, seen = [], set()
for p, cyc in early_candidates:
    if p.exists() and cyc not in seen:
        df = read_any(p)
        df["SDDSRVYR"] = cyc
        early_parts.append(df)
        seen.add(cyc)
fsq_early_raw = pd.concat(early_parts, ignore_index=True) if early_parts else pd.DataFrame()

# B) Main consolidated (03–18)
if not fsq_main_path.exists():
    raise FileNotFoundError(f"FSQS file not found: {fsq_main_path}")
fsq_main_raw = read_any(fsq_main_path)

# C) Filter to adults using your helper
fsq_early = filter_adults(fsq_early_raw) if not fsq_early_raw.empty else fsq_early_raw
fsqs       = filter_adults(fsq_main_raw)

# D) Stack early + main
fsq_all = pd.concat([fsq_early, fsqs], ignore_index=True)
fsq_all.columns = fsq_all.columns.str.upper()

# E) Build SNAP/FS outputs (guards for varying columns)
snap = pd.DataFrame({"SEQN": fsq_all["SEQN"]})
if "SDDSRVYR" in fsq_all: snap["SDDSRVYR"] = fsq_all["SDDSRVYR"]
if "FSDHH" in fsq_all:   snap["FSDHH"] = fsq_all["FSDHH"]

snap["SNAP"] = np.nan
if "FSQ165" in fsq_all: snap.loc[fsq_all["FSQ165"] == 2, "SNAP"] = 0
if "FSQ012" in fsq_all:
    snap.loc[fsq_all["FSQ012"] == 1, "SNAP"] = 1
    snap.loc[fsq_all["FSQ012"] == 2, "SNAP"] = 0
if "FSQ171" in fsq_all:
    snap.loc[fsq_all["FSQ171"] == 1, "SNAP"] = 1
    snap.loc[fsq_all["FSQ171"] == 2, "SNAP"] = 0
if "FSD170N" in fsq_all: snap.loc[fsq_all["FSD170N"] >= 1, "SNAP"] = 1
if "FSQ170" in fsq_all:
    snap.loc[fsq_all["FSQ170"] == 1, "SNAP"] = 1
    snap.loc[(fsq_all["FSQ170"] == 2) & (fsq_all.get("FSD170N", pd.Series(index=fsq_all.index)) < 1), "SNAP"] = 0
if "FSD200" in fsq_all: snap.loc[fsq_all["FSD200"] == 1, "SNAP"] = 1

snap["FS"] = np.nan
if "FSDHH" in fsq_all:
    snap.loc[fsq_all["FSDHH"].isin([1, 2]), "FS"] = 1
    snap.loc[fsq_all["FSDHH"] > 2, "FS"] = 0

# Final columns
snap = snap[[c for c in ["SEQN", "SNAP", "FSDHH", "FS", "SDDSRVYR"] if c in snap.columns]]

# Hygiene
snap["SEQN"] = pd.to_numeric(snap["SEQN"], errors="coerce").astype("Int64")
if "SDDSRVYR" in snap.columns:
    snap["SDDSRVYR"] = pd.to_numeric(snap["SDDSRVYR"], errors="coerce").astype("Int64")
snap = snap.dropna(subset=["SEQN"]).drop_duplicates(subset=["SEQN"])

# Quick checks
print("SNAP table shape:", snap.shape)
if "SDDSRVYR" in snap.columns:
    print("Cycles present:\n", snap["SDDSRVYR"].value_counts().sort_index())

print(snap["SEQN"])


SNAP table shape: (55081, 5)
Cycles present:
 SDDSRVYR
1    4880
2    5411
Name: count, dtype: Int64
0             2
1             5
2             7
3            10
4            12
          ...  
55076    102950
55077    102952
55078    102953
55079    102954
55080    102956
Name: SEQN, Length: 55081, dtype: Int64


<h3> HEI? COV </h3>

In [62]:
# Step 1: Read dietary score data
scores = pd.read_excel(os.path.join(folder_path, "i.scores.xlsx"), engine="openpyxl")

# Rename columns to match desired output
scores = scores.rename(columns={
    "seqn": "SEQN",
    "i.FCS": "i_FCS",
    "i.optup": "i_optup",  # keep lowercase here first
    "i.HSR": "i_HSR",
    "i.nutri": "i_nutri"
})

# Then copy and rename for output
scores2 = scores[["SEQN", "i_FCS", "i_optup", "i_HSR", "i_nutri"]].copy()
scores2 = scores2.rename(columns={"i_optup": "i_Optup"})
scores2 = scores2.sort_values("SEQN")


# Step 2: Read covariates from Lu paper
covar = pd.read_sas(os.path.join(folder_path, "covar.sas7bdat"), format="sas7bdat")
covar = covar.rename(columns=str.upper)  # make all column names uppercase to match SAS style
# filter available variables only
covar_vars = ["SEQN", "RIDAGEYR", "SEX", "RACE", "EDU", "INDFMPIR", "SMK_AVG", "SMK_PAST",
              "SMK", "ALCG2", "HEI2015_TOTAL_SCORE", "DIABE"]
covar = covar[[col for col in covar_vars if col in covar.columns]].copy()
covar = covar.sort_values("SEQN")

# Step 3: Read covariates from Meghan paper
covariates1_raw = pd.read_csv(os.path.join(folder_path, "covariates.csv"))
covariates1 = covariates1_raw.rename(columns={"seqn": "SEQN"})
covariates_vars = ["SEQN", "sdmvpsu", "sdmvstra", "met_hr", "perE_alco", "dm_self",
                   "tchol", "hdl", "ldl", "tg", "bmi", "CVD", "dm_rx", "chol_rx",
                   "angina_rx", "lung_disease", "angina", "hba1c", "sbp", "dbp", "cancer"]
covariates1 = covariates1[[col for col in covariates_vars if col in covariates1.columns]].copy()
covariates1 = covariates1.sort_values("SEQN")

# Step 4: Read dietary weight data (filter DAYS == 1)
dietwt = pd.read_sas(os.path.join(folder_path, "gg.sas7bdat"), format="sas7bdat")

# Check for expected columns
required_cols = ["SEQN", "DAYS", "WTDRD1", "WTDR2D", "DR12DRST"]
missing = [col for col in required_cols if col not in dietwt.columns]
if missing:
    print(f"Warning: Missing columns from gg.sas7bdat: {missing}")

# Filter and select
dietwt = dietwt[dietwt["DAYS"] == 1][["SEQN", "WTDRD1", "WTDR2D", "DR12DRST"]].copy()
dietwt = dietwt.sort_values("SEQN")


# Step 5: Read mortality data
mort = pd.read_sas(os.path.join(folder_path, "mortality9918.sas7bdat"), format="sas7bdat")
mort = mort.sort_values("SEQN")

def summarize_df(name, df):
    print(f"{name}:")
    print(f"  Rows: {df.shape[0]}")
    print(f"  Unique SEQN: {df['SEQN'].nunique()}")
    print("-" * 40)

summarize_df("scores2", scores2)
summarize_df("covar", covar)
summarize_df("covariates1", covariates1)
summarize_df("dietwt", dietwt)
summarize_df("mort", mort)

scores2:
  Rows: 39746
  Unique SEQN: 39746
----------------------------------------
covar:
  Rows: 101316
  Unique SEQN: 101316
----------------------------------------
covariates1:
  Rows: 39262
  Unique SEQN: 39262
----------------------------------------
dietwt:
  Rows: 88413
  Unique SEQN: 88413
----------------------------------------
mort:
  Rows: 59064
  Unique SEQN: 59064
----------------------------------------


In [103]:

#  SEQN check — i.scores, covar, covariates, gg (dietwt), mortality9918

import pandas as pd
import numpy as np
import os
from pathlib import Path

# Set your folder
folder_path = "/Users/dengshuyue/Desktop/SDOH/analysis/data/less_important"

def _standardize_seqn(df):
    # make columns case-insensitive and return a Series of SEQN (nullable Int64)
    df.columns = df.columns.str.upper()
    if "SEQN" not in df.columns:
        return None
    s = pd.to_numeric(df["SEQN"], errors="coerce").astype("Int64")
    return s

def summarize_seqn(name, seqn):
    print(f"\n{name}")
    if seqn is None:
        print("  ❌ SEQN column not found")
        return
    print(f"  Rows: {len(seqn)}")
    print(f"  Null SEQN: {seqn.isna().sum()}")
    if seqn.notna().any():
        print(f"  dtype: {seqn.dtype}")
        print(f"  Min / Max: {seqn.min()} / {seqn.max()}")
        nunq = seqn.nunique(dropna=True)
        dups = (seqn.notna().sum() - nunq)
        print(f"  Unique SEQN: {nunq}  |  Duplicates: {dups}")
        # small sample
        print(f"  Head SEQN: {list(seqn.dropna().head(5))}")
        print(f"  Tail SEQN: {list(seqn.dropna().tail(5))}")

# 1) i.scores.xlsx
try:
    scores = pd.read_excel(os.path.join(folder_path, "i.scores.xlsx"), engine="openpyxl")
    # handle seqn casing
    if "seqn" in scores.columns and "SEQN" not in scores.columns:
        scores = scores.rename(columns={"seqn": "SEQN"})
    summarize_seqn("i.scores.xlsx", _standardize_seqn(scores))
except Exception as e:
    print("\n[i.scores.xlsx] ⚠️", e)

# 2) covar.sas7bdat
try:
    covar = pd.read_sas(os.path.join(folder_path, "covar.sas7bdat"),
                        format="sas7bdat", encoding="latin1")
    summarize_seqn("covar.sas7bdat", _standardize_seqn(covar))
except Exception as e:
    print("\n[covar.sas7bdat] ⚠️", e)

# 3) covariates.csv
try:
    covariates1_raw = pd.read_csv(os.path.join(folder_path, "covariates.csv"))
    if "seqn" in covariates1_raw.columns and "SEQN" not in covariates1_raw.columns:
        covariates1_raw = covariates1_raw.rename(columns={"seqn": "SEQN"})
    summarize_seqn("covariates.csv", _standardize_seqn(covariates1_raw))
except Exception as e:
    print("\n[covariates.csv] ⚠️", e)

# 4) gg.sas7bdat (diet weights)
try:
    dietwt = pd.read_sas(os.path.join(folder_path, "gg.sas7bdat"),
                         format="sas7bdat", encoding="latin1")
    summarize_seqn("gg.sas7bdat", _standardize_seqn(dietwt))
except Exception as e:
    print("\n[gg.sas7bdat] ⚠️", e)

# 5) mortality9918.sas7bdat
try:
    mort = pd.read_sas(os.path.join(folder_path, "mortality9918.sas7bdat"),
                       format="sas7bdat", encoding="latin1")
    summarize_seqn("mortality9918.sas7bdat", _standardize_seqn(mort))
except Exception as e:
    print("\n[mortality9918.sas7bdat] ⚠️", e)



i.scores.xlsx
  Rows: 39746
  Null SEQN: 0
  dtype: Int64
  Min / Max: 21009 / 102956
  Unique SEQN: 39746  |  Duplicates: 0
  Head SEQN: [np.int64(21009), np.int64(21010), np.int64(21012), np.int64(21015), np.int64(21017)]
  Tail SEQN: [np.int64(102949), np.int64(102952), np.int64(102953), np.int64(102954), np.int64(102956)]

covar.sas7bdat
  Rows: 101316
  Null SEQN: 0
  dtype: Int64
  Min / Max: 1 / 102956
  Unique SEQN: 101316  |  Duplicates: 0
  Head SEQN: [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5)]
  Tail SEQN: [np.int64(102952), np.int64(102953), np.int64(102954), np.int64(102955), np.int64(102956)]

covariates.csv
  Rows: 39262
  Null SEQN: 0
  dtype: Int64
  Min / Max: 21009 / 102956
  Unique SEQN: 39262  |  Duplicates: 0
  Head SEQN: [np.int64(21009), np.int64(21010), np.int64(21012), np.int64(21015), np.int64(21017)]
  Tail SEQN: [np.int64(102947), np.int64(102952), np.int64(102953), np.int64(102954), np.int64(102956)]

gg.sas7bdat
  Rows: 149358
  Nul

In [ ]:
# 🔥🔥🔥🔥🔥🔥 NOW WORK AT HERE!!!!!!
# try to extend covariates.csv to 99-18 currently is 03-18



In [63]:
print("RIDAGEYR" in demoall.columns)  # Should print True
print(demoall.columns)

True
Index(['SEQN', 'RIDAGEYR', 'marriage', 'SDDSRVYR'], dtype='object')


In [64]:
# Ensure SEQN is int in all datasets
# for name, df in datasets.items():
#    df["SEQN"] = df["SEQN"].astype(int)

In [65]:
# Rename RIDAGEYR in covar to avoid overwriting demoall's RIDAGEYR
covar = covar.rename(columns={"RIDAGEYR": "RIDAGEYR_covar"})


In [66]:
from functools import reduce

# ✅ Step 0: Merge all datasets by SEQN
merge_list = [demoall, hoq_all, ocq, mort, snap, ins, covar, dietwt, scores2, covariates1]
score_mort = reduce(lambda left, right: pd.merge(left, right, on="SEQN", how="left"), merge_list)

In [67]:
# ✅ Step 1: Filter to adults age ≥ 20
if "RIDAGEYR" not in score_mort.columns:
    raise ValueError("RIDAGEYR is missing after merge — check if demoall was merged correctly.")
score_mort = score_mort[score_mort["RIDAGEYR"] >= 20]

In [68]:
# Step 2: Variable transformations
score_mort["wt10"] = score_mort["WTDRD1"] / 10
score_mort["wt"] = score_mort["WTDR2D"] / 8
score_mort["i_FCS_sd"] = score_mort["i_FCS"] / 10.89
score_mort["i_Optup_sd"] = score_mort["i_Optup"] / 8.17
score_mort["i_nutri_sd"] = -score_mort["i_nutri"] / 3.17
score_mort["i_HSR_sd"] = score_mort["i_HSR"] / 1.01
score_mort["hei2015_sd"] = score_mort["HEI2015_TOTAL_SCORE"] / 13

# Step 3: Recode death indicators
for cause, code in {
    "death_heart": "001", "death_cancer": "002", "death_resp": "003", "Death_inj": "004",
    "death_cerev": "005", "Death_alz": "006", "death_diabe": "007",
    "Death_infl": "008", "Death_kid": "009", "death_other1": "010"
}.items():
    score_mort[cause] = (score_mort["UCOD_LEADING"] == code).astype(int)

# Step 4: Composite categories
score_mort["Death_other"] = score_mort[["death_resp", "Death_inj", "Death_alz", "Death_infl"]].sum(axis=1).clip(upper=1)
score_mort["Death_oth2"] = score_mort[["death_resp", "Death_inj", "Death_alz", "Death_infl", "death_other1"]].sum(axis=1).clip(upper=1)
score_mort["death_cvd"] = score_mort[["death_heart", "death_cerev"]].sum(axis=1).clip(upper=1)
score_mort["death_cmd"] = score_mort[["death_heart", "death_cerev", "death_diabe"]].sum(axis=1).clip(upper=1)
score_mort["death_cmdk"] = score_mort[["death_heart", "death_cerev", "death_diabe", "Death_kid"]].sum(axis=1).clip(upper=1)
score_mort["death_cmdkh"] = score_mort["death_cmdk"]
score_mort.loc[score_mort["DIABETES"] == 1, "death_cmdkh"] = 1
score_mort.loc[score_mort["HYPERTEN"] == 1, "death_cmdkh"] = 1
score_mort["death_cmd"] = score_mort["death_cmd"].fillna(0)
score_mort.loc[score_mort["death_cmd"] == 1, ["Death_other", "Death_oth2"]] = 0

# Step 5: Multiple cause mortality
score_mort["death_multi"] = score_mort["MORTSTAT"]
score_mort.loc[score_mort["death_cmd"] == 1, "death_multi"] = 1
score_mort.loc[score_mort["death_cancer"] == 1, "death_multi"] = 2
score_mort.loc[score_mort["Death_oth2"] == 1, "death_multi"] = 3

# Step 6: Age & time vars
score_mort["agesq"] = score_mort["RIDAGEYR"] ** 2
score_mort["py"] = score_mort["PERMTH_EXM"] / 12
score_mort["agestart"] = score_mort["RIDAGEYR"]
score_mort["ageend"] = score_mort["RIDAGEYR"] + score_mort["py"]

# Step 7: Poverty
score_mort["pir"] = 5
score_mort.loc[(score_mort["INDFMPIR"] < 1.3) & (score_mort["INDFMPIR"].notna()), "pir"] = 1
score_mort.loc[(score_mort["INDFMPIR"] >= 1.3), "pir"] = 2
score_mort.loc[(score_mort["INDFMPIR"] >= 3), "pir"] = 3

# Step 8: Recode SNAP
score_mort.loc[(score_mort["INDFMPIR"].between(0, 1.3)) & (score_mort["SNAP"] != 1), "SNAP"] = 2

# Step 9: BMI categories
score_mort["bmic"] = pd.NA
score_mort.loc[(score_mort["bmi"] > 0) & (score_mort["bmi"] < 18.5), "bmic"] = 0
score_mort.loc[(score_mort["bmi"] >= 18.5) & (score_mort["bmi"] < 25), "bmic"] = 1
score_mort.loc[(score_mort["bmi"] >= 25), "bmic"] = 2
score_mort.loc[(score_mort["bmi"] >= 30), "bmic"] = 3

In [69]:
# check drop by step

# Step 1: Initial inclusion based on ELIGSTAT and MORTSTAT
score_mort["include"] = np.where(
    (score_mort["ELIGSTAT"] == 1) & (score_mort["MORTSTAT"].notna()), 1, 0
)
print(f"After ELIGSTAT & MORTSTAT inclusion: {score_mort['include'].value_counts().to_dict()}")

# Step 2: Exclude missing HEI, or DR12DRST > 1 
step2_cond = (
    (score_mort["include"] == 1) &
    (
        score_mort["HEI2015_TOTAL_SCORE"].isna() |
        #(score_mort["WTDRD1"] <= 0) |
        (score_mort["DR12DRST"] > 1)
    )
)
print(f"Dropped at Step 2 (diet data or recall quality): {step2_cond.sum()}")
score_mort.loc[step2_cond, "include"] = 2

# Step 3: Exclude missing FS, SNAP, or pir == 4
step3_cond = (
    (score_mort["include"] == 1) &
    (
        score_mort["FS"].isna() |
        score_mort["SNAP"].isna() |
        (score_mort["pir"] == 4)
    )
)
print(f"Dropped at Step 3 (missing FS/SNAP or pir=4): {step3_cond.sum()}")
score_mort.loc[step3_cond, "include"] = 3

# Step 4: Exclude if WTDRD1 <= 0 
step4_cond = (score_mort["include"] == 1) & (score_mort["WTDRD1"] <= 0)
print(f"Dropped at Step 4 (zero or negative WTDRD1): {step4_cond.sum()}")
score_mort.loc[step4_cond, "include"] = 4

# Final count of each inclusion code
print(f"📊 Final include flag counts: {score_mort['include'].value_counts().to_dict()}")

# Apply final filter
score_mort = score_mort[score_mort["include"] == 1].copy()
print(f" Final analytic sample size: {score_mort.shape[0]}")


# Inclusion flag
score_mort["include"] = np.where(
    (score_mort["ELIGSTAT"] == 1) & (score_mort["MORTSTAT"].notna()), 1, 0
)
score_mort.loc[
    (score_mort["include"] == 1) &
    (
        (score_mort["HEI2015_TOTAL_SCORE"].isna()) |
        (score_mort["WTDRD1"] <= 0) |
        (score_mort["DR12DRST"] > 1)
    ),
    "include"
] = 2
score_mort.loc[
    (score_mort["include"] == 1) &
    (
        score_mort["FS"].isna() |
        score_mort["SNAP"].isna() |
        (score_mort["pir"] == 4)
    ),
    "include"
] = 3
score_mort.loc[
    (score_mort["include"] == 1) & (score_mort["WTDRD1"] <= 0),
    "include"
] = 4

# Insurance binary
score_mort["ins2"] = np.where(score_mort["ins"] == 0, 0, 1)

# Unemployment indicator
score_mort["unemployment2"] = np.where(score_mort["employ"] > 1, 1, 0)

# Final filter to keep only those with include == 1
score_mort = score_mort[score_mort["include"] == 1].copy()

# Save final dataset
score_mort.to_pickle(os.path.join(folder_path, "SODH_diet_mort.pkl"))  # You can also use .csv or .parquet

After ELIGSTAT & MORTSTAT inclusion: {1: 54945, 0: 136}
Dropped at Step 2 (diet data or recall quality): 6288
Dropped at Step 3 (missing FS/SNAP or pir=4): 9755
Dropped at Step 4 (zero or negative WTDRD1): 0
📊 Final include flag counts: {1: 38902, 3: 9755, 2: 6288, 0: 136}
 Final analytic sample size: 38902


<h3>check final merged data </h3>

In [32]:
SODH_diet_mort = pd.read_pickle(os.path.join(folder_path, "SODH_diet_mort.pkl"))

score_mort.to_csv("/Users/dengshuyue/Desktop/SDOH/analysis/data/SODH_diet_mort_depr2.csv", index=False)

# Number of rows
total_rows = SODH_diet_mort.shape[0]

# Number of unique SEQN values
unique_ids = SODH_diet_mort['SEQN'].nunique()

print(f" Total rows: {total_rows}")
print(f" Unique SEQN values: {unique_ids}")


 Total rows: 38902
 Unique SEQN values: 38902


In [33]:
# Check age range
min_age = score_mort["RIDAGEYR"].min()
max_age = score_mort["RIDAGEYR"].max()

print(f" Age range: {min_age} to {max_age}")

 Age range: 20.0 to 85.0


In [34]:
# View column names
print(SODH_diet_mort.columns.tolist())

['SEQN', 'RIDAGEYR', 'marriage', 'SDDSRVYR', 'HOD050', 'HOQ065', 'employ', 'unemployment', 'ELIGSTAT', 'MORTSTAT', 'UCOD_LEADING', 'DIABETES', 'HYPERTEN', 'PERMTH_INT', 'PERMTH_EXM', 'Death_heart', 'Death_cancer', 'Death_resp', 'Death_cerev', 'Death_diabe', 'Death_other', 'death_cvd', 'death_cmd', 'SNAP', 'FSDHH', 'FS', 'ins', 'RIDAGEYR_covar', 'SEX', 'RACE', 'EDU', 'INDFMPIR', 'SMK_AVG', 'SMK', 'ALCG2', 'HEI2015_TOTAL_SCORE', 'DIABE', 'WTDRD1', 'WTDR2D', 'DR12DRST', 'i_FCS', 'i_Optup', 'i_HSR', 'i_nutri', 'sdmvpsu', 'sdmvstra', 'met_hr', 'perE_alco', 'dm_self', 'tchol', 'hdl', 'ldl', 'tg', 'bmi', 'CVD', 'dm_rx', 'chol_rx', 'angina_rx', 'lung_disease', 'angina', 'hba1c', 'sbp', 'dbp', 'cancer', 'wt10', 'wt', 'i_FCS_sd', 'i_Optup_sd', 'i_nutri_sd', 'i_HSR_sd', 'hei2015_sd', 'death_heart', 'death_cancer', 'death_resp', 'Death_inj', 'death_cerev', 'Death_alz', 'death_diabe', 'Death_infl', 'Death_kid', 'death_other1', 'Death_oth2', 'death_cmdk', 'death_cmdkh', 'death_multi', 'agesq', 'py',

<h1>Demographic Table</h1>

<h2>🧮 Generate Weighted Demographic Summary Table</h2>

<h3>try demo all item </h3>

In [281]:
import pandas as pd

# Load the .pkl file
pkl_path = "/Users/dengshuyue/Desktop/SDOH/analysis/data/SODH_diet_mort.pkl"
df = pd.read_pickle(pkl_path)

# Save as .csv
csv_path = "/Users/dengshuyue/Desktop/SDOH/analysis/data/SODH_diet_mort.csv"
df.to_csv(csv_path, index=False)

print("✅ Conversion complete: .pkl → .csv")


✅ Conversion complete: .pkl → .csv


In [279]:
# Python does not support full survey design (strata + PSU) out of the box
# below code not working 
# === Load data ===
data_path = "/Users/dengshuyue/Desktop/SDOH/analysis/data/"
df = pd.read_pickle(os.path.join(data_path, "SODH_diet_mort.pkl"))

# Drop rows missing survey design variables
df = df.dropna(subset=['wt10', 'sdmvstra', 'sdmvpsu'])

# === Weighted proportions for categorical variables ===
def weighted_props(df, var, weight):
    d = df[[var, weight]].dropna()
    counts = d.groupby(var)[weight].sum()
    total_weight = d[weight].sum()
    props = (counts / total_weight).round(3)
    return counts.round(0).astype(int), (props * 100).round(1)

# === Use Taylor Linearization for SEs of continuous variables ===
from linearmodels.survey import SurveyDesign, SurveyMean

def survey_mean_se(df, var, weight='wt10', strata='sdmvstra', cluster='sdmvpsu'):
    # Remove missing
    d = df[[var, weight, strata, cluster]].dropna()

    # Define survey design
    design = sm.survey.SurveyDesign(
        strata=d[strata],
        cluster=d[cluster],
        weights=d[weight],
    )

    # Fit the design-based estimator
    survey_var = sm.survey.SurveyMean(d[var], design)
    return float(survey_var.mean), float(survey_var.std)

# === Variable dictionaries ===
cat_vars = {
    'SEX': {1: 'Male', 2: 'Female'},
    'RACE': {1: 'Non-Hispanic White', 2: 'Non-Hispanic Black', 3: 'Hispanic', 4: 'Other'},
    'EDU': {1: 'Less than high school', 2: 'High school or equivalent', 3: 'Some college', 4: 'College or above'},
    'pir': {1: '<1.3', 2: '1.3~2.99', 3: '>=3'},
    'SNAP': {0: 'Not participant', 1: 'Participant', 2: 'Income eligible non-participant'},
    'SMK': {1: 'Nonsmokers', 2: 'Former smokers', 3: '<15 cigarettes/day', 4: '15-24.9 cigarettes/day', 5: '≥ 25 cigarettes/day'},
    'ALCG2': {1: 'Nondrinkers', 2: 'Moderate drinker', 3: 'Heavy drinker', 4: 'Missing'},
    'bmic': {1: 'BMI <18.5', 2: '18-24.9', 3: '25-29.9', 4: 'BMI ≥30'}
}

binary_vars = ['DIABETES', 'CVD', 'dm_rx', 'chol_rx', 'angina', 'cancer', 'lung_disease', 'MORTSTAT']
cont_vars = ['RIDAGEYR', 'met_hr', 'bmi', 'hba1c', 'sbp', 'dbp', 'hdl', 'ldl', 'tg', 'HEI2015_TOTAL_SCORE']

# === Generate summary ===
rows = []

# Categorical
for var, mapping in cat_vars.items():
    counts, props = weighted_props(df, var, 'wt10')
    for code, label in mapping.items():
        if code in counts.index:
            rows.append({
                "Variable": var,
                "Category": label,
                "Overall": f"{counts[code]:,.0f} ({props[code]}%)"
            })

# Binary (weighted %)
for var in binary_vars:
    val, se = survey_mean_se(df, var)
    rows.append({
        "Variable": var,
        "Category": "1",
        "Overall": f"{val * 100:.1f}% ({se * 100:.1f})"
    })

# Continuous (mean ± SE)
for var in cont_vars:
    mean, se = survey_mean_se(df, var)
    rows.append({
        "Variable": var,
        "Category": "",
        "Overall": f"{mean:.2f} ({se:.2f})"
    })

# === Create summary table ===
demo_table = pd.DataFrame(rows)

# === Save and preview ===
output_path = os.path.join(data_path, "demo_summary.csv")
demo_table.to_csv(output_path, index=False)

# Show preview
demo_table.head(50)


ModuleNotFoundError: No module named 'linearmodels.survey'

In [264]:

# === Load data ===
df = pd.read_pickle(os.path.join(data_path, "SODH_diet_mort.pkl"))

# Drop missing survey design variables (weight pooled: WTDRD1)
df = df.dropna(subset=['wt10', 'sdmvstra', 'sdmvpsu'])

# === Helper functions ===
def weighted_mean(x, w):
    d = pd.DataFrame({'x': x, 'w': w}).dropna()
    return np.sum(d.x * d.w) / np.sum(d.w)

# not standard 
# def weighted_se(x, w):
#    d = pd.DataFrame({'x': x, 'w': w}).dropna()
#    mean = weighted_mean(d.x, d.w)
#    return np.sqrt(np.sum(d.w * (d.x - mean)**2) / ((len(d) - 1) * np.sum(d.w) / len(d)))

# Only acceptable if weights are uniform
# def weighted_se(x, w):
#    d = pd.DataFrame({'x': x, 'w': w}).dropna()
#    mean = np.sum(d.x * d.w) / np.sum(d.w)
#    var = np.sum(d.w * (d.x - mean)**2) / np.sum(d.w)
#    se = np.sqrt(var / len(d))  # Approximate
#    return se

# 🔥 Check best practice for estimating SE with survey weights

def weighted_se(x, w):
    d = pd.DataFrame({'x': x, 'w': w}).dropna()
    mean = np.sum(d.x * d.w) / np.sum(d.w)
    eff_n = (np.sum(d.w))**2 / np.sum(d.w**2)  # Effective sample size
    var = np.sum(d.w * (d.x - mean)**2) / np.sum(d.w)
    se = np.sqrt(var / eff_n)
    return se

    
def weighted_props(df, var, weight):
    d = df[[var, weight]].dropna()
    counts = d.groupby(var)[weight].sum()
    total_weight = d[weight].sum()
    props = (counts / total_weight).round(3)
    return counts.round(0).astype(int), (props * 100).round(1)

# === Categorical variables ===
cat_vars = {
    'SEX': {1: 'Male', 2: 'Female'},
    'RACE': {1: 'Non-Hispanic White', 2: 'Non-Hispanic Black', 3: 'Hispanic', 4: 'Other'},
    'EDU': {1: 'Less than high school', 2: 'High school or equivalent', 3: 'Some college', 4: 'College or above'},
    'pir': {1: '<1.3', 2: '1.3~2.99', 3: '>=3'},
    'SNAP': {0: 'Not participant', 1: 'Participant', 2: 'Income eligible non-participant'},
    'SMK': {1: 'Nonsmokers', 2: 'Former smokers', 3: '<15 cigarettes/day', 4: '15-24.9 cigarettes/day', 5: '≥ 25 cigarettes/day'},
    'ALCG2': {1: 'Nondrinkers', 2: 'Moderate drinker', 3: 'Heavy drinker', 4: 'Missing'},
    'bmic': {1: 'BMI <18.5', 2: '18-24.9', 3: '25-29.9', 4: 'BMI ≥30'}
}

rows = []

for var, mapping in cat_vars.items():
    counts, props = weighted_props(df, var, 'wt10')
    for code, label in mapping.items():
        if code in counts.index:
            rows.append({
                "Variable": var,
                "Category": label,
                "Overall": f"{counts[code]:,.0f} ({props[code]}%)"
            })

# === Binary variables ===
binary_vars = ['DIABETES', 'CVD', 'dm_rx', 'chol_rx', 'angina', 'cancer', 'lung_disease', 'MORTSTAT']
for var in binary_vars:
    val = weighted_mean(df[var].fillna(0), df['wt10']) * 100
    rows.append({
        "Variable": var,
        "Category": "1",
        "Overall": f"{val:.1f}%"
    })

# === Continuous variables ===
cont_vars = ['RIDAGEYR', 'met_hr', 'bmi', 'hba1c', 'sbp', 'dbp', 'hdl', 'ldl', 'tg', 'HEI2015_TOTAL_SCORE']
for var in cont_vars:
    mean = weighted_mean(df[var], df['wt10'])
    se = weighted_se(df[var], df['wt10'])
    rows.append({
        "Variable": var,
        "Category": "",
        "Overall": f"{mean:.2f} ({se:.2f})"
    })

# === Create summary table ===
demo_table = pd.DataFrame(rows)

# Save, preview

# Optionally: save to CSV
# demo_table.to_csv("demo_summary.csv", index=False)
demo_table.to_csv(output_path, index=False)

demo_table.head(50)

,Variable,Category,Overall
0,SEX,Male,"82,147,536 (47.8%)"
1,SEX,Female,"89,889,015 (52.2%)"
2,RACE,Non-Hispanic White,"117,066,666 (68.0%)"
3,RACE,Non-Hispanic Black,"19,270,383 (11.2%)"
4,RACE,Hispanic,"23,166,597 (13.5%)"
5,RACE,Other,"12,532,906 (7.3%)"
6,EDU,Less than high school,"27,365,333 (15.9%)"
7,EDU,High school or equivalent,"41,028,422 (23.9%)"
8,EDU,Some college,"54,476,932 (31.7%)"
9,EDU,College or above,"49,075,426 (28.5%)"


In [227]:
# Clean data: drop rows with missing mortality status or weight
mort_df = df[['MORTSTAT', 'WTDRD1']].dropna()

# Calculate total weighted sum
total_weight = mort_df['WTDRD1'].sum()

# Weighted percentage of deceased (MORTSTAT == 1)
dead_weight = mort_df.loc[mort_df['MORTSTAT'] == 1, 'WTDRD1'].sum()
weighted_pct_dead = (dead_weight / total_weight) * 100

print(f"Weighted percentage deceased: {weighted_pct_dead:.1f}%")


Weighted percentage deceased: 9.1%


In [22]:
# Define unweighted counts and proportions
def unweighted_props(df, var):
    counts = df[var].value_counts(dropna=False).sort_index()
    props = counts / counts.sum() * 100
    return counts, props.round(1)

# Now use it for 'sex'
counts, props = unweighted_props(df, 'sex')

# Optionally map codes to labels
sex_labels = {1: 'Male', 2: 'Female'}
for val in counts.index:
    label = sex_labels.get(val, val)
    print(f"{label}: {counts[val]} ({props[val]}%)")
18530+19890


Male: 18845 (48.4%)
Female: 20055 (51.6%)


38420

In [ ]:
with open('/Users/dengshuyue/Desktop/SDOH/analysis/code/Ref/2.Prepare data_master.sas', 'r', encoding='latin1') as f:
    sas_code = f.read()

print(sas_code[:20000])  # Preview the first X,000 characters

In [ ]:
with open('/Users/dengshuyue/Desktop/SDOH/analysis/code/Ref/2.1_Prepare data_covariates.sas', 'r', encoding='latin1') as f:
    sas_code = f.read()

print(sas_code[:20000])  # Preview the first X,000 characters

In [ ]:
with open('/Users/dengshuyue/Desktop/SDOH/analysis/code/Descriptions_1.sas', 'r', encoding='latin1') as f:
    sas_code = f.read()

print(sas_code[:20000])  # Preview the first 15,000 characters

In [ ]:
# Diabetes Identification Note:
# The final diabetes indicator variable used in the analysis is 'diabe2'.
# This composite variable identifies diabetes based on the following criteria:
# - Self-reported physician diagnosis (DIQ010), current insulin use (DIQ050), or oral medication use (DIQ070)
# - Prescription drug data indicating diabetes treatment (dm_rx2)
# - Fasting glucose ≥ 126 mg/dL (glu_dm)
# - OGTT ≥ 200 mg/dL (ogtt_dm)
# - HbA1c ≥ 6.5% (hb_dm)
# Any one of these criteria being met will set 'diabe2' = 1, otherwise 0.


In [ ]:
with open('/Users/dengshuyue/Desktop/SDOH/analysis/Analysis1_COX_allcause_bysub.sas', 'r', encoding='latin1') as f:
    sas_code = f.read()

print(sas_code[:10000])  # Preview the first 15,000 characters

In [ ]:
with open('/Users/dengshuyue/Desktop/SDOH/analysis/Analysis1_dataprep.sas', 'r', encoding='latin1') as f:
    sas_code = f.read()

print(sas_code[:20000])  # Preview the first 15,000 characters

<h1>SDOH score</h1>

In [20]:
# see housing and employ
df[['HOQ065', 'HOD050', 'EMPLOY']]

,HOQ065,HOD050,EMPLOY
0,1.0,6.0,1.0
1,1.0,3.0,2.0
2,1.0,7.0,3.0
3,1.0,6.0,3.0
4,2.0,3.0,1.0
...,...,...,...
38895,3.0,8.0,1.0
38896,1.0,6.0,3.0
38897,2.0,1.0,1.0
38898,2.0,8.0,2.0


In [21]:
# List of selected SDOH-related variables
selected_cols = ['HOQ065', 'HOD050', 'MOR']

# Summary function for each variable
for col in selected_cols:
    print(f"\n=== {col} Summary ===")
    print("Value counts:")
    print(df[col].value_counts(dropna=False))
    print("\nProportions:")
    print((df[col].value_counts(normalize=True, dropna=False) * 100).round(2))
    print(f"\nMissing values: {df[col].isnull().sum()}")


=== HOQ065 Summary ===
Value counts:
HOQ065
1.0    24162
2.0    13779
3.0      907
NaN       52
Name: count, dtype: int64

Proportions:
HOQ065
1.0    62.11
2.0    35.42
3.0     2.33
NaN     0.13
Name: proportion, dtype: float64

Missing values: 52

=== HOD050 Summary ===
Value counts:
HOD050
5.0      8034
6.0      7469
4.0      6416
7.0      5175
8.0      3352
3.0      3193
9.0      1908
10.0     1122
2.0       820
11.0      476
12.0      285
1.0       281
13.0      260
999.0      43
777.0      39
NaN        27
Name: count, dtype: int64

Proportions:
HOD050
5.0      20.65
6.0      19.20
4.0      16.49
7.0      13.30
8.0       8.62
3.0       8.21
9.0       4.90
10.0      2.88
2.0       2.11
11.0      1.22
12.0      0.73
1.0       0.72
13.0      0.67
999.0     0.11
777.0     0.10
NaN       0.07
Name: proportion, dtype: float64

Missing values: 27

=== EMPLOY Summary ===
Value counts:
EMPLOY
1.0    20703
3.0     7765
5.0     4323
4.0     3634
2.0     1594
NaN      881
Name: count, dtype: